## 🔹 1) Установка Airflow на Debian 

Все команды вводятся в **терминале Debian**.

### 1.1 Поставить базовые пакеты
```bash
sudo apt update
sudo apt install -y python3 python3-venv python3-pip
```


### 1.2 Создать папку под окружение Airflow и виртуальное окружение
```bash
mkdir -p ~/ml_airflow
cd ~/ml_airflow

python3 -m venv .venv
source .venv/bin/activate

pip install -U pip
```

Проверка, что ты “внутри” venv:
```bash
which python
python --version
```

### 1.3 Настроить AIRFLOW_HOME (где будут конфиги, логи и папка dags)
Airflow использует `~/airflow` по умолчанию; переменная `AIRFLOW_HOME` задаёт, где Airflow хранит свои файлы.
```bash
export AIRFLOW_HOME=~/airflow
mkdir -p "$AIRFLOW_HOME"
```

### 1.4 Установить Airflow через pip с constraints 
Документация Quick Start показывает установку с constraint‑файлом вида `constraints-${PYTHON_VERSION}.txt` по URL шаблону.

```bash
AIRFLOW_VERSION=3.1.3
PYTHON_VERSION="$(python -c 'import sys; print(f"{sys.version_info.major}.{sys.version_info.minor}")')"
CONSTRAINT_URL="https://raw.githubusercontent.com/apache/airflow/constraints-${AIRFLOW_VERSION}/constraints-${PYTHON_VERSION}.txt"

pip install "apache-airflow==${AIRFLOW_VERSION}" --constraint "${CONSTRAINT_URL}"
```

Проверка:
```bash
airflow version
```

### 1.5 Поставить то, чем будем исполнять ноутбук
Чтобы Airflow мог запускать `.ipynb`, удобно использовать papermill (он сохраняет “отработанный” ноутбук отдельным файлом).  
В том же venv:
```bash
pip install papermill jupyter
```

***

## 🔹 2) Куда класть ipynb/датасет и как сохранять результаты

### 2.1 Создать структуру проекта и папку результатов на Desktop
```bash
mkdir -p ~/ml_project/notebooks
mkdir -p ~/ml_project/data
mkdir -p ~/Desktop/c_m
```

Куда что класть:
- `~/ml_project/notebooks/train.ipynb` — ноутбук, который обучает модель и считает метрики.
- `~/ml_project/data/` — датасет(ы), которые ноутбук читает.
- `~/Desktop/c_m/` — сюда утром должны попасть:
  - выполненный ноутбук,
  - папка с моделью и метриками.

### 2.2 Настрка ноутбука на сохранение в output_dir
Смысл: Airflow передаст параметр `output_dir`, а ноутбук будет сохранять результаты туда.

В начале ноутбука необходимо сделать параметр (papermill его переопределит):
```python
output_dir = "/tmp/airflow_default"
```

Дальше сохраняем в коде:
- метрики → `output_dir/metrics.json`
- модель → `output_dir/model.pt` 

Пример:
```python
from pathlib import Path
import json

out = Path(output_dir)
out.mkdir(parents=True, exist_ok=True)

# ... train ...

metrics = {"accuracy": 0.93, "f1": 0.90}
(out / "metrics.json").write_text(json.dumps(metrics, indent=2), encoding="utf-8")

# пример под PyTorch:
# torch.save(model.state_dict(), out / "model.pt")
```

***

## 🔹 3) DAG “только на одну ночь” и сохранение в `~/Desktop/c_m`

### 3.1 Создать DAG-файл (куда класть)
Airflow подхватывает DAG‑файлы из папки `"$AIRFLOW_HOME/dags"` (то есть обычно `~/airflow/dags`).

Открыть файл:
```bash
nano ~/airflow/dags/one_night_train_to_c_m.py
```

Вставить (заменить `<USER>` на имя пользователя в `/home/<USER>/...`):
```python
from datetime import datetime
from airflow import DAG
from airflow.operators.bash import BashOperator

USER = "<USER>"

NOTEBOOK_IN = f"/home/{USER}/ml_project/notebooks/train.ipynb"
OUT_ROOT = f"/home/{USER}/Desktop/c_m"

with DAG(
    dag_id="one_night_train_to_c_m",
    start_date=datetime(2025, 1, 1),
    schedule=None,          # <-- ключ: НЕТ расписания, запуск ТОЛЬКО вручную
    catchup=False,
    tags=["ml", "one-off"],
) as dag:

    # Папка под конкретный запуск 
    prepare_dir = BashOperator(
        task_id="prepare_dir",
        bash_command=f"mkdir -p {OUT_ROOT}/{{{{ ds_nodash }}}}",
    )

    # Выполняем ноутбук и сохраняем "executed" ipynb в корень OUT_ROOT
    # Модель/метрики ноутбук сохранит в OUT_ROOT/<ds_nodash>/
    run_notebook = BashOperator(
        task_id="run_notebook",
        bash_command=f"""
        papermill {NOTEBOOK_IN} \
          {OUT_ROOT}/executed_train_{{{{ ds_nodash }}}}.ipynb \
          -p output_dir {OUT_ROOT}/{{{{ ds_nodash }}}}
        """,
    )

    # Проверяем, что результаты действительно появились
    check_outputs = BashOperator(
        task_id="check_outputs",
        bash_command=f"""
        test -f {OUT_ROOT}/{{{{ ds_nodash }}}}/metrics.json
        test -f {OUT_ROOT}/{{{{ ds_nodash }}}}/model.pt
        """,
    )

    prepare_dir >> run_notebook >> check_outputs
```


***

## 🔹 4) Как запустить Airflow 

### Вечером  — подготовка и старт
#### Шаг 1 — откроем терминал №1 и активируем окружение
```bash
cd ~/ml_airflow
source .venv/bin/activate
export AIRFLOW_HOME=~/airflow
```

#### Шаг 2 — запускаем Airflow
```bash
airflow standalone
```
`airflow standalone` инициализирует БД, создаёт пользователя и запускает компоненты; UI открывается на `http://localhost:8080`, а логин/пароль админа показываются в терминале.

Важно на ночь:
- **не закрываем** терминал №1, пока обучение не закончится (он должен работать всю ночь).

#### Шаг 3 — зайти в UI и запустить DAG вручную
1) Открыть браузер: `http://localhost:8080`.
2) Войти логином/паролем, которые увидим в терминале после `airflow standalone`.
3) На странице DAG‑ов найти `one_night_train_to_c_m`.  
4) Перевести DAG в состояние “активен/не на паузе” (unpause).  
5) Нажать **Trigger DAG** (ручной запуск). Airflow в этот момент создаст DagRun и начнёт выполнять задачи.

(Если что-то упадёт — в UI можно открыть логи упавшей таски и понять причину, не трогая ноутбук.)

### Ночь:
- Будет выполняться DagRun: сначала `prepare_dir`, затем `run_notebook`, затем `check_outputs`.
- В `~/Desktop/c_m/` начнут появляться артефакты, но финальная структура будет после успешного завершения.

### Утром:
- Выполненный ноутбук: `~/Desktop/c_m/executed_train_YYYYMMDD.ipynb`
- Модель и метрики: `~/Desktop/c_m/YYYYMMDD/model.pt` и `~/Desktop/c_m/YYYYMMDD/metrics.json`

Когда всё готово — можно остановить Airflow в терминале №1: `Ctrl+C`.

***

## 🔹 5) Альтернатива: запуск вручную без UI (CLI)
Airflow позволяет вручную триггерить DagRun из командной строки.

В новом терминале (при активированном venv и `AIRFLOW_HOME`):
```bash
cd ~/ml_airflow
source .venv/bin/activate
export AIRFLOW_HOME=~/airflow

airflow dags trigger one_night_train_to_c_m
```
Это создаст DagRun так же, как кнопка Trigger в UI.

***
